____
__Universidad Tecnológica Nacional, Buenos Aires__<br/>
__Ingeniería Industrial__<br/>
__Cátedra de Ciencia de Datos - Curso I5521__<br/>
__Elaborado por: Martin Palazzo__
____

# Clasificación en dataset Breast Cancer Wisconsin
En el siguiente notebook aplicaremos distintos clasificadores sobre un popular dataset de cancer de mamas. Analizaremos distintas formas de entrenar los modelos y utilizaremos distintos enfoques para evaluar los resultados y seleccionar los hiper-parametros.

#### **Contexto del dataset**
Supongamos que tenemos un set de datos de $n=569$ muestras de tumores, cada una caracterizada por 32 variables de las cuales $d = 31$ de ellas son variables observadas en cada tumor y almacenadas en cada registro. La variable restante $y$ es el diagnostico Benigno (B) o Maligno (M) etiquetado por el especialista en patología. <br>
<br>
Vamos a querer aprender una regla de decisión utilizando métodos de Machine Learning que decida si un tumor es Maligno o Benigno unicamente observando las 31 variables.<br>
<br>
Es decir que aprenderemos una funcion $f(x) = y$ donde $x$ son las variables independientes que caracterizan a cada tumor (features), la función $f(x)$ será utilizada para decidir la categoría $y$ usada para representar la etiqueta del tumor.

Importar las librerias que venimos usando siempre.

In [ ]:
# importamos las librerías necesarias para trabajar.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

Importamos librerias de scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import (
    cross_validate,
    train_test_split,
)

### Importamos dataset de Wisconsin Breast Cancer

In [ ]:
pwd

In [ ]:
ls

In [ ]:
# importamos el dataset con Pandas
# el parámetro index_col indica que la primer columna (la 0) contiene un indicador único por cáda registro
# index_col no será parte del dataset, solo cumplirá la función de índice.
breast = pd.read_csv('dataset_breast_wisconsin.csv', delimiter=';', index_col = 0)


Visualizamos el dataset con Pandas. Observamos que en la columna "diagnosis" se encuentra si el tumor es benigno o maligno. El resto de las columnas refiere a distintas features del tumor. Ver que la primer columna es el ID del paciente, esta información no nos interesa!!

In [ ]:
breast.head()

In [ ]:
# observamos el tamaño del dataset
breast.shape

# Pre-procesamiento del dataset

Vamos a revisar si existen valores nulos en la tabla que contiene los datos.

In [ ]:
# el comando .isnull() de Pandas revisará cada columna para ver si existen o no valores nulos.
breast.isnull().any()

**Separo mi variable independiente**: Vamos a guardar en una matriz llamada $x$ las features asociadas a cada muestra. Esta matriz contendra datos que utilizaremos para predecir la cateogira (Maligno-Benigno) de un tumor. Las filas de la matriz representarán cada tumor mientras que las columnas representarán las variables/features.

Inicialmente vamos a crear "x" solamente utilizando las features desde la columna 2 a la 4. Recordemos que en el dataset la columna "diagnosis" no es una variable independiente y solo representaa las etiquetas.

In [ ]:
# En esta primera instancia vamos a usar las features 10, 11 , 12 y 13.Para ello realizaremos un filtrado de pandas con .iloc
# en un "slice" el último número no cuenta.
x = breast.iloc[:,10:14]

In [ ]:
# con .head() observamos el dataframe resultante
x.head()

In [ ]:
# realizamos un heatmap sobre la matriz X resultante 
sns.set_style("white")
sns.set_context("talk")
sns.set_style("ticks")
sns.heatmap(x)
plt.show()

Con un Pairplot vamos a visualizar como se relacionan par-a-par las variables seleccionadas a lo largo de todo el dataset.

In [ ]:
sns.set_style("white")
sns.set_context("talk")
sns.set_style("ticks")
sns.pairplot(x)
plt.show()

In [ ]:
# vamos a calcular la correlacion lineal de pearson para las features de nuestro dataset
corr_matrix = x.corr()

In [ ]:
# ademas vamos a visualizar la matriz de correlacion
sns.set_style("white")
sns.set_context("talk")
sns.set_style("ticks")
sns.heatmap( corr_matrix ,annot=True)
plt.show()

**Separo mi variable dependiente Y**: Ahora procederemos a crear el vector de etiquetas con las variables dependientes. Ver que las etiquetas se encuentran en la 1er columna (es decir la columna 0) del dataset.

In [ ]:
# con .iloc filtramos la primer columna del dataframe para obtener las etiquetas
y = breast.iloc[:,0]

In [ ]:
# las etiquetas/labels estan presentadas como categorias y deberiamos pasarlas a una representacion numerica
y.head()

**Transformo mi variable dependiente Y en numerica**: previamente estaba como string.
Vemos que las etiquetas se encuentran asignadas con M y B. Para transformarlas a numéricas utilizaremos el "LabelEncoder" de sklearn y las pasaremos a numeros.

In [ ]:
from sklearn import preprocessing

# "le" es el label encoder que transforma las etiquetas de string a INT.
le = preprocessing.LabelEncoder()
# sobre-escribo el vector "y" con las etiquetas numericas obtenidas del label encoder
y=le.fit_transform(y)

In [ ]:
# ahora las etiquetas son numericas
y

**Separo mi dataset en Train y Test**:
Con la funcion "train_test_split" de sklearn separaremos nuestro dataset (tanto x como y) en dos sets de entrenamiento y prueba independiente. Podemos elegir que % de muestras esten en cada conjunto. El random_state sirve para que pueda reproducirse la "aleatoriedad" de division de muestras en el futuro. 

In [ ]:
# obtenemos xtrain, xtest, ytrain e ytest :) 
# OJO! en este caso estamos usando 95% de test para hacerla mas dificil
# en general el test set es entre 20 y 30% 

xtrain, xtest, ytrain, ytest = train_test_split(x.values, y, test_size=0.95, random_state=4)

In [ ]:
# observamos como queda la matriz de xtrain
xtrain.shape

In [ ]:
# observamos como queda la matriz de xtest
xtest.shape

**Auto-Scaling utilizando muestras de train**: 
Queremos dejar todas las features en los mismos rangos por eso utilizaremos el standard scaler para que queden con media 0 y desvio standard 1.

In [ ]:
# auto scaling train- set (mean = 0, std = 1)
# ver que en la misma linea creamos el standard scaler y lo "fiteamos" al mismo tiempo con "xtrain"
scaler = preprocessing.StandardScaler().fit(xtrain)

In [ ]:
# xtrain_scal tendra el dataset de train pre-procesado con el standard scaler
xtrain_scal = scaler.transform(xtrain)  

In [ ]:
# para cada feature imprimimos la media y el desvio standard obtenido. Ver que media = 0 y stdev = 1.
print("media de cada feature a lo largo del dataset")
print(xtrain_scal.mean(axis=0))
print(" ")
print("Desvio STD de cada feature a lo largo del dataset")
print(xtrain_scal.std(axis=0))

In [ ]:
# observamos en que type quedo xtrain_xcal
type(xtrain_scal)

In [ ]:
# utilizando el scaler "fiteado" o "ajustado" a los datos de train, aplicamos el scaler a los datos de test.
# obtenemos "xtest_scal" 
xtest_scal = scaler.transform(xtest)  

In [ ]:
# obviamente la media y stdev de las features en test no seran 0 y 1 ya que el scaler se ajusto con train unicamente.
print(xtest_scal.mean(axis=0))
print(xtest_scal.std(axis=0))

Visualizamos la matriz de datos de entrenamiento previo y post a estandarizacion/preprocesamiento con un mapa de calor.

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16,5))
sns.heatmap(x, ax = axs[0])
sns.heatmap(xtrain_scal, ax = axs[1])
sns.heatmap(xtest_scal, ax = axs[2])
axs[0].set_title('Raw data pre-standarization')
axs[1].set_title('Standarized train data')
axs[2].set_title('Standarized test data')

Visualizamos la distribucion de las features con boxplots de entrenamiento previo a estandarizacion/preprocesamiento con un boxplot por cada variable.

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(10,4))
#plt.figure(figsize=(14,8))
axs[0].boxplot([xtrain[:,0],xtrain[:,1],xtrain[:,2], xtrain[:,3]])
axs[1].boxplot([xtrain_scal[:,0],xtrain_scal[:,1],xtrain_scal[:,2], xtrain_scal[:,3]])
axs[0].set_title('Variables pre estandarizacion')
axs[1].set_title('Variables post estandarizacion')
plt.show()

## Entrenar modelo con Logistic Regression

Defino modelo LR

In [ ]:
# defino modelo LR llamado model_lr
# (OJO! no estamos haciendo grid-search)
model_lr = LogisticRegression()


In [ ]:
# realizo un cross validation de 5 folds para ver la performance del modelo con distintas particiones de entrenamiento
cv_results = cross_validate(model_lr, xtrain_scal, ytrain, cv=5)

In [ ]:
# analizamos los resultados de cada iteracion del cross validation. 
# Este objeto indica el tiempo que tomo cada iteracion, y en "test_score" cuanto dio el resultado de accuracy
cv_results

In [ ]:
# estudiamos como fue el promedio de todos los accuracys a lo largo de todas las iteraciones de cross validation
np.mean(cv_results['test_score'])

Ajusto-entreno (fiteo) modelo a los datos de train. OJO! aca no estamos usando cross-validation. 

In [ ]:
# Ajusto mi modelo a las muestras de training utilizando tanto Xtrain como Ytrain
model_lr.fit(xtrain_scal, ytrain) 

Hago prediccion de los datos de test (sin utilizar las etiquetas).

In [ ]:
# Genero predicciones con mi modelo entrenado para las muestras de Test, utilizando solo Xtest
ypred = model_lr.predict(xtest_scal)

In [ ]:
# visualizamos en pantalla el resultado de las predicciones para cada muestra del conjunto de test
ypred

Computo la exactitud de mi modelo entre las etiquetas de test reales y las asignadas por el modelo.

In [ ]:
# Computo el accuracy (comparar ytest vs ypred)
test_acc = accuracy_score(ytest, ypred)
print("El accuracy es " + str(test_acc))

Computamos el Area debajo de la curva ROC del modelo.

In [ ]:
#compute and plot AUC
yproba = model_lr.predict_proba(xtest_scal)
fpr1, tpr1, thresholds = roc_curve(ytest.astype('int'), yproba[:,1], drop_intermediate = False)
auc_ = auc(fpr1, tpr1)
print("El AUC es = " + str(auc_))

Imprimimos el AUC ROC

In [ ]:
plt.plot(fpr1, tpr1, lw=2, alpha=0.7 , label = 'ROC curve', color = 'b')
plt.plot([0, 1], [0, 1], linestyle='--', lw=1, color='r',label='Luck', alpha=.8)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(False)
plt.legend(loc="lower right")
plt.title('ROC curve with LR')
plt.show()

Calculamos la matriz de confusión.

In [ ]:
# Compute confusion Matrix
cm = confusion_matrix(ytest, ypred)
cm

In [ ]:
df_cm = pd.DataFrame(cm, index = ['Cancer', 'Control'], columns = ['Cancer', 'Control'])
plt.figure(figsize = (6,4))
sns.heatmap(df_cm, annot=True)
plt.title('Classification Confusion matrix')
plt.show()

# Tarea 01: 
- crear un nuevo set de xtrain y xtest utilizando todas las variables disponibles ya que antes solo habiamos usado un sub-conjunto de las mismas(y asi tener mejor poder de prediccion :)
- Deberan nombrarlas con otro nombre distinto al creado anteriormente y luego comparar resultados de AUC ROC y Accuracy.
- utilizar 70% de train y 30% test.

In [ ]:
######### CODIGO AQUI ############
######### CODIGO AQUI ############
######### CODIGO AQUI ############
######### CODIGO AQUI ############

# Tarea 02: 
- Un SVM con grid search y cross validation cv = 5 utilizando el dataset con todas las features realizado en el punto anterior. 
- Comparar resultados de clasificación entre ambos modelos.
- determinar que parametros son los "ganadores" del grid search para cada modelo.

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10, 100, 1000], 'gamma':[0.0001,0.001, 0.01, 0.1,1, 10,100]}
svc = svm.SVC()
clf_svm = GridSearchCV(svc, param_grid = parameters, refit = True, cv = 5)

In [ ]:
######### CODIGO AQUI ############
######### CODIGO AQUI ############
######### CODIGO AQUI ############
######### CODIGO AQUI ############

# Tarea 03:
- Entrenar un modelo de logistic regression sobre los datos del punto 01 aplicando cross validation.
- comparar los resultados con el SVM.

In [ ]:
from sklearn.linear_model import LogisticRegression

parameters = {'C':[1, 10, 100, 1000]}
lr_model = LogisticRegression(random_state=0, solver='lbfgs')
clf_lr = GridSearchCV(lr_model, param_grid = parameters, refit = True, cv = 5)

In [ ]:
######### CODIGO AQUI ############
######### CODIGO AQUI ############
######### CODIGO AQUI ############
######### CODIGO AQUI ############

## Tarea 04: Learning courve
¿que sucede si probamos distintos tamaños de train set? ¿como se modificara el accuracy y AUC?
Probar con trainset = <br>
- 5%
- 10%
- 15%
- 20%
- 25%
- ..%
- ..%
- 100%
y comparar para KNN, LR y SVM.

In [ ]:
######### CODIGO AQUI ############
######### CODIGO AQUI ############
######### CODIGO AQUI ############
######### CODIGO AQUI ############

In [ ]:
######### CODIGO AQUI ############
######### CODIGO AQUI ############
######### CODIGO AQUI ############
######### CODIGO AQUI ############

In [ ]:
######### CODIGO AQUI ############
######### CODIGO AQUI ############
######### CODIGO AQUI ############
######### CODIGO AQUI ############